In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans,DBSCAN
import tensorflow as tf

from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV,RepeatedStratifiedKFold

from sklearn.preprocessing import StandardScaler,PowerTransformer
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline

2023-08-26 19:50:00.428165: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-08-26 19:50:00.428289: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
data=pd.read_csv('diabetes.csv',index_col=0)
data.head(5)

,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
Pregnancies,,,,,,,,
6,148,72,35,0,33.6,0.627,50,1
1,85,66,29,0,26.6,0.351,31,0
8,183,64,0,0,23.3,0.672,32,1
1,89,66,23,94,28.1,0.167,21,0
0,137,40,35,168,43.1,2.288,33,1


In [5]:
data.isnull().sum()

Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [6]:
y_full=data['Outcome']
X_full=data.drop(['Outcome'],axis=1)
print(X_full.shape)
print(y_full.shape)

(768, 7)
(768,)


In [7]:

X_train,X_test,y_train,y_test=train_test_split(X_full,y_full,test_size=0.2,random_state=0)
print(X_train.shape)
print(X_test.shape)

(614, 7)
(154, 7)


In [8]:

def get_score(model):
    model.fit(X_train,y_train)
    predicts=model.predict(X_test)
    error=mean_absolute_error(y_test,predicts)
    return error

In [11]:

def model_comparison():
    mod = {}
   
    scaler = StandardScaler()
    power = PowerTransformer(method = 'yeo-johnson')
    
    clf1 = RandomForestClassifier() 
    clf2= XGBClassifier() 
    clf3=DecisionTreeClassifier()
    clf4=KMeans(n_clusters=2)
    clf6 = LogisticRegression()

    mod['Random Forest'] = Pipeline(steps=[('scale', scaler), ('pow', power),('mod', clf1)])
    mod['XGBoost'] = Pipeline(steps=[('scale', scaler), ('pow', power),('mod', clf2)])
    mod['Desicion Tree']=Pipeline(steps=[('scale',scaler),('pow',power),('mod',clf3)])
    mod['kmean']=Pipeline(steps=[('scale',scaler),('mod',clf4)])
    mod['Logistic Regression'] = Pipeline(steps=[('scale', scaler), ('pow', power),('mod', clf6)])

    return mod
my_model=model_comparison()
print('don')


don


In [12]:
def cross_val(mod):
    scores = cross_val_score(mod, X_train, y_train, cv=5, scoring='f1_weighted')
    return scores


for name, model in my_model.items():
    scores = cross_val(model)
    print(f'{name}: F1-weighted score:', scores.mean())


Random Forest: F1-weighted score: 0.74528213607988
XGBoost: F1-weighted score: 0.7151159416275408
Desicion Tree: F1-weighted score: 0.68116051284306
kmean: F1-weighted score: 0.41827971756036575
Logistic Regression: F1-weighted score: 0.7564668456713652


In [22]:

model1=LogisticRegression()

In [23]:
logistic_params={
    'penalty':['l1','l2'],
    'C':[0.1,0.5,0.01,0.05,1,10],
    'solver':['liblinear','lbfgs','saga'],
    'max_iter':[100,200,300]
    
}

grid_search_log=GridSearchCV(model1,logistic_params,cv=5,scoring='f1_weighted',verbose=2)
grid_search_log.fit(X_train,y_train)
print('Best Parameters :', grid_search_log.best_params_)
print('F1 score :', grid_search_log.best_score_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
[CV] END ..C=0.1, max_iter=100, penalty=l1, solver=liblinear; total time=   0.0s
[CV] END ..C=0.1, max_iter=100, penalty=l1, solver=liblinear; total time=   0.0s
[CV] END ..C=0.1, max_iter=100, penalty=l1, solver=liblinear; total time=   0.0s
[CV] END ..C=0.1, max_iter=100, penalty=l1, solver=liblinear; total time=   0.0s
[CV] END ..C=0.1, max_iter=100, penalty=l1, solver=liblinear; total time=   0.0s
[CV] END ......C=0.1, max_iter=100, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ......C=0.1, max_iter=100, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ......C=0.1, max_iter=100, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ......C=0.1, max_iter=100, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ......C=0.1, max_iter=100, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .......C=0.1, max_iter=100, penalty=l1, solver=saga; total time=   0.0s
[CV] END .......C=0.1, max_iter=100, penalty=l

/home/jude/.local/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/jude/.local/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/jude/.local/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/jude/.local/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/jude/.local/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/jude/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: Convergenc

[CV] END ......C=0.1, max_iter=100, penalty=l2, solver=lbfgs; total time=   0.0s
[CV] END ......C=0.1, max_iter=100, penalty=l2, solver=lbfgs; total time=   0.0s
[CV] END ......C=0.1, max_iter=100, penalty=l2, solver=lbfgs; total time=   0.0s
[CV] END ......C=0.1, max_iter=100, penalty=l2, solver=lbfgs; total time=   0.0s
[CV] END ......C=0.1, max_iter=100, penalty=l2, solver=lbfgs; total time=   0.0s
[CV] END .......C=0.1, max_iter=100, penalty=l2, solver=saga; total time=   0.0s
[CV] END .......C=0.1, max_iter=100, penalty=l2, solver=saga; total time=   0.0s
[CV] END .......C=0.1, max_iter=100, penalty=l2, solver=saga; total time=   0.0s
[CV] END .......C=0.1, max_iter=100, penalty=l2, solver=saga; total time=   0.0s
[CV] END .......C=0.1, max_iter=100, penalty=l2, solver=saga; total time=   0.0s
[CV] END ..C=0.1, max_iter=200, penalty=l1, solver=liblinear; total time=   0.0s
[CV] END ..C=0.1, max_iter=200, penalty=l1, solver=liblinear; total time=   0.0s
[CV] END ..C=0.1, max_iter=2

/home/jude/.local/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/jude/.local/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/jude/.local/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/jude/.local/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/jude/.local/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/jude/.local/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarn

[CV] END .......C=0.1, max_iter=200, penalty=l1, solver=saga; total time=   0.0s
[CV] END .......C=0.1, max_iter=200, penalty=l1, solver=saga; total time=   0.0s
[CV] END .......C=0.1, max_iter=200, penalty=l1, solver=saga; total time=   0.0s
[CV] END .......C=0.1, max_iter=200, penalty=l1, solver=saga; total time=   0.0s
[CV] END .......C=0.1, max_iter=200, penalty=l1, solver=saga; total time=   0.0s
[CV] END ..C=0.1, max_iter=200, penalty=l2, solver=liblinear; total time=   0.0s
[CV] END ..C=0.1, max_iter=200, penalty=l2, solver=liblinear; total time=   0.0s
[CV] END ..C=0.1, max_iter=200, penalty=l2, solver=liblinear; total time=   0.0s
[CV] END ..C=0.1, max_iter=200, penalty=l2, solver=liblinear; total time=   0.0s
[CV] END ..C=0.1, max_iter=200, penalty=l2, solver=liblinear; total time=   0.0s
[CV] END ......C=0.1, max_iter=200, penalty=l2, solver=lbfgs; total time=   0.0s
[CV] END ......C=0.1, max_iter=200, penalty=l2, solver=lbfgs; total time=   0.0s
[CV] END ......C=0.1, max_it

...

/home/jude/.local/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/jude/.local/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/jude/.local/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/jude/.local/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/jude/.local/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV] END ...C=10, max_iter=300, penalty=l1, solver=liblinear; total time=   0.0s
[CV] END ...C=10, max_iter=300, penalty=l1, solver=liblinear; total time=   0.0s
[CV] END .......C=10, max_iter=300, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .......C=10, max_iter=300, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .......C=10, max_iter=300, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .......C=10, max_iter=300, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .......C=10, max_iter=300, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ........C=10, max_iter=300, penalty=l1, solver=saga; total time=   0.0s
[CV] END ........C=10, max_iter=300, penalty=l1, solver=saga; total time=   0.0s
[CV] END ........C=10, max_iter=300, penalty=l1, solver=saga; total time=   0.0s
[CV] END ........C=10, max_iter=300, penalty=l1, solver=saga; total time=   0.0s


/home/jude/.local/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/jude/.local/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/jude/.local/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/jude/.local/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV] END ........C=10, max_iter=300, penalty=l1, solver=saga; total time=   0.0s
[CV] END ...C=10, max_iter=300, penalty=l2, solver=liblinear; total time=   0.0s
[CV] END ...C=10, max_iter=300, penalty=l2, solver=liblinear; total time=   0.0s
[CV] END ...C=10, max_iter=300, penalty=l2, solver=liblinear; total time=   0.0s
[CV] END ...C=10, max_iter=300, penalty=l2, solver=liblinear; total time=   0.0s
[CV] END ...C=10, max_iter=300, penalty=l2, solver=liblinear; total time=   0.0s
[CV] END .......C=10, max_iter=300, penalty=l2, solver=lbfgs; total time=   0.0s
[CV] END .......C=10, max_iter=300, penalty=l2, solver=lbfgs; total time=   0.0s
[CV] END .......C=10, max_iter=300, penalty=l2, solver=lbfgs; total time=   0.0s
[CV] END .......C=10, max_iter=300, penalty=l2, solver=lbfgs; total time=   0.0s
[CV] END .......C=10, max_iter=300, penalty=l2, solver=lbfgs; total time=   0.0s
[CV] END ........C=10, max_iter=300, penalty=l2, solver=saga; total time=   0.0s
[CV] END ........C=10, max_i

/home/jude/.local/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/jude/.local/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/jude/.local/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/jude/.local/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/jude/.local/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/jude/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFa

In [46]:
best_log_reg_model=Pipeline(steps=[
    ('scale',StandardScaler()),
    ('pow',PowerTransformer(method = 'yeo-johnson')),
    ('mod',LogisticRegression(C=1,max_iter=100,penalty='l2',solver='lbfgs'))
])
scores=cross_val_score(best_log_reg_model,X_train,y_train,cv=5,scoring='f1_weighted')
print(scores.mean())

0.7564668456713652


In [26]:
get_score(best_log_reg_model)

0.22077922077922077

In [44]:
my_pipeline=Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('pow',PowerTransformer(method='yeo-johnson')),
    ('clf1', XGBClassifier(n_estimators=1200,learning_rate=0.01))
])
scores=cross_val_score(my_pipeline,X_train,y_train,
                      cv=5,
                      scoring='f1_weighted')

print(scores.mean())

0.717060940732486


In [43]:
get_score(my_pipeline)

0.18831168831168832

In [47]:
random_for_params={
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2'],
    'bootstrap': [True, False]
}
random_forest=RandomForestClassifier()
random_grid_search=GridSearchCV(random_forest,random_for_params,cv=5,scoring='f1_weighted',verbose=2)
random_grid_search.fit(X_train,y_train)
print("best params :", random_grid_search.best_params_)
print("best score :", random_grid_search.best_score_)

Fitting 5 folds for each of 324 candidates, totalling 1620 fits
[CV] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.2s
[CV] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.2s
[CV] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.2s
[CV] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.2s
[CV] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.2s
[CV] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.3s
[CV] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estim

[CV] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=300; total time=   0.6s
[CV] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=100; total time=   0.2s
[CV] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=100; total time=   0.2s
[CV] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=100; total time=   0.2s
[CV] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=100; total time=   0.2s
[CV] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=100; total time=   0.2s
[CV] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=200; total time=   0.4s
[CV] END bootstrap=True, max_depth

[CV] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=300; total time=   0.6s
[CV] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=100; total time=   0.2s
[CV] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=100; total time=   0.2s
[CV] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=100; total time=   0.2s
[CV] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=100; total time=   0.2s
[CV] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=100; total time=   0.2s
[CV] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=200; total time=   0.3s
[CV] END bootstrap=True, max

[CV] END bootstrap=True, max_depth=None, max_features=log2, min_samples_leaf=1, min_samples_split=10, n_estimators=300; total time=   0.5s
[CV] END bootstrap=True, max_depth=None, max_features=log2, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   0.2s
[CV] END bootstrap=True, max_depth=None, max_features=log2, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   0.2s
[CV] END bootstrap=True, max_depth=None, max_features=log2, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   0.2s
[CV] END bootstrap=True, max_depth=None, max_features=log2, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   0.2s
[CV] END bootstrap=True, max_depth=None, max_features=log2, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   0.2s
[CV] END bootstrap=True, max_depth=None, max_features=log2, min_samples_leaf=2, min_samples_split=2, n_estimators=200; total time=   0.3s
[CV] END bootstrap=True, max_dept

[CV] END bootstrap=True, max_depth=None, max_features=log2, min_samples_leaf=4, min_samples_split=2, n_estimators=300; total time=   0.4s
[CV] END bootstrap=True, max_depth=None, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=100; total time=   0.1s
[CV] END bootstrap=True, max_depth=None, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=100; total time=   0.2s
[CV] END bootstrap=True, max_depth=None, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=100; total time=   0.2s
[CV] END bootstrap=True, max_depth=None, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=100; total time=   0.2s
[CV] END bootstrap=True, max_depth=None, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=100; total time=   0.2s
[CV] END bootstrap=True, max_depth=None, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=200; total time=   0.3s
[CV] END bootstrap=True, max_depth

...

[CV] END bootstrap=False, max_depth=10, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=200; total time=   0.3s
[CV] END bootstrap=False, max_depth=10, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=200; total time=   0.3s
[CV] END bootstrap=False, max_depth=10, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=200; total time=   0.3s
[CV] END bootstrap=False, max_depth=10, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=200; total time=   0.3s
[CV] END bootstrap=False, max_depth=10, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=300; total time=   0.4s
[CV] END bootstrap=False, max_depth=10, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=300; total time=   0.4s
[CV] END bootstrap=False, max_depth=10, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=300; total time=   0.4s
[CV] END bootstrap=False, max_depth=10, m

[CV] END bootstrap=False, max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   0.3s
[CV] END bootstrap=False, max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   0.3s
[CV] END bootstrap=False, max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   0.3s
[CV] END bootstrap=False, max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   0.3s
[CV] END bootstrap=False, max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=300; total time=   0.4s
[CV] END bootstrap=False, max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=300; total time=   0.4s
[CV] END bootstrap=False, max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=300; total time=   0.4s
[CV] END bootstrap=False, max_dept

[CV] END bootstrap=False, max_depth=20, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=200; total time=   0.3s
[CV] END bootstrap=False, max_depth=20, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=200; total time=   0.3s
[CV] END bootstrap=False, max_depth=20, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=200; total time=   0.3s
[CV] END bootstrap=False, max_depth=20, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=200; total time=   0.3s
[CV] END bootstrap=False, max_depth=20, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=300; total time=   0.4s
[CV] END bootstrap=False, max_depth=20, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=300; total time=   0.4s
[CV] END bootstrap=False, max_depth=20, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=300; total time=   0.4s
[CV] END bootstrap=False, max_depth=20, m

[CV] END bootstrap=False, max_depth=20, max_features=log2, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   0.3s
[CV] END bootstrap=False, max_depth=20, max_features=log2, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   0.3s
[CV] END bootstrap=False, max_depth=20, max_features=log2, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   0.3s
[CV] END bootstrap=False, max_depth=20, max_features=log2, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   0.3s
[CV] END bootstrap=False, max_depth=20, max_features=log2, min_samples_leaf=1, min_samples_split=5, n_estimators=300; total time=   0.5s
[CV] END bootstrap=False, max_depth=20, max_features=log2, min_samples_leaf=1, min_samples_split=5, n_estimators=300; total time=   0.4s
[CV] END bootstrap=False, max_depth=20, max_features=log2, min_samples_leaf=1, min_samples_split=5, n_estimators=300; total time=   0.5s
[CV] END bootstrap=False, max_depth=20, m

[CV] END bootstrap=False, max_depth=20, max_features=log2, min_samples_leaf=2, min_samples_split=10, n_estimators=200; total time=   0.3s
[CV] END bootstrap=False, max_depth=20, max_features=log2, min_samples_leaf=2, min_samples_split=10, n_estimators=200; total time=   0.3s
[CV] END bootstrap=False, max_depth=20, max_features=log2, min_samples_leaf=2, min_samples_split=10, n_estimators=200; total time=   0.3s
[CV] END bootstrap=False, max_depth=20, max_features=log2, min_samples_leaf=2, min_samples_split=10, n_estimators=200; total time=   0.3s
[CV] END bootstrap=False, max_depth=20, max_features=log2, min_samples_leaf=2, min_samples_split=10, n_estimators=300; total time=   0.4s
[CV] END bootstrap=False, max_depth=20, max_features=log2, min_samples_leaf=2, min_samples_split=10, n_estimators=300; total time=   0.4s
[CV] END bootstrap=False, max_depth=20, max_features=log2, min_samples_leaf=2, min_samples_split=10, n_estimators=300; total time=   0.4s
[CV] END bootstrap=False, max_dept

In [50]:
final_random_model=pipeline=Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('pow',PowerTransformer(method='yeo-johnson')),
    ('best_model', RandomForestClassifier(n_estimators=100,
                                          bootstrap=True,max_features='log2',min_samples_leaf=1,min_samples_split=5))
])
scores=cross_val_score(my_pipeline,X_train,y_train,
                      cv=5,
                      scoring='f1_weighted')

print(scores.mean())

0.717060940732486


In [51]:
get_score(final_model)

0.2012987012987013

In [53]:
final_model=best_log_reg_model


NotFittedError: This StandardScaler instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.